In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import itertools
import time

In [2]:
# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load the data
# Replace 'your_data.csv' with your actual file path
df = pd.read_csv('df_final.csv')

# Split into training (2024 and before) and testing (2025)
train_df = df[df['year_2025'] == 0]
test_df = df[df['year_2025'] == 1]

In [3]:
np.random.seed(42)
tf.random.set_seed(42)

# Load the data
# Replace 'your_data.csv' with your actual file path
df = pd.read_csv('df_final.csv')

# Split into training (2024 and before) and testing (2025)
train_df = df[df['year_2025'] == 0]
test_df = df[df['year_2025'] == 1]

In [4]:
# Further split training data into training and validation
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Separate features and target
X_train = train_df.drop('monthly_rent', axis=1)
y_train = train_df['monthly_rent']
X_val = val_df.drop('monthly_rent', axis=1)
y_val = val_df['monthly_rent']
X_test = test_df.drop('monthly_rent', axis=1)
y_test = test_df['monthly_rent']

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

print(f"Training features shape: {X_train_scaled.shape}")
print(f"Validation features shape: {X_val_scaled.shape}")
print(f"Testing features shape: {X_test_scaled.shape}")

Training features shape: (115772, 36)
Validation features shape: (28944, 36)
Testing features shape: (5980, 36)


In [5]:
# Define hyperparameter grid for manual search
hyperparams = {
    'neurons_layer1': [64, 128],
    'neurons_layer2': [32, 64],
    'neurons_layer3': [16, 32],
    'dropout_rate1': [0.2, 0.3],
    'dropout_rate2': [0.1, 0.2],
    'learning_rate': [0.01, 0.001],
    'activation': ['relu'],
    'batch_size': [32, 64]
}

In [6]:
def create_model(input_dim, neurons_layer1, neurons_layer2, neurons_layer3,
                dropout_rate1, dropout_rate2, learning_rate, activation):
    model = Sequential([
        Dense(neurons_layer1, activation=activation, input_dim=input_dim),
        Dropout(dropout_rate1),
        Dense(neurons_layer2, activation=activation),
        Dropout(dropout_rate2),
        Dense(neurons_layer3, activation=activation),
        Dense(1)  # Output layer (no activation for regression)
    ])

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='mean_squared_error'
    )

    return model

In [ ]:
keys = list(hyperparams.keys())
values = list(hyperparams.values())
hyperparameter_combinations = list(itertools.product(*values))

# Results storage
results = []

# Start timing
start_time = time.time()

# Total number of combinations
total_combinations = len(hyperparameter_combinations)
print(f"Testing {total_combinations} hyperparameter combinations...")

# Early stopping for each model
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Manual grid search
for i, params in enumerate(hyperparameter_combinations):
    # Create a dictionary of hyperparameters
    params_dict = dict(zip(keys, params))

    # Print progress
    print(f"\nTraining model {i+1}/{total_combinations} with parameters:")
    for key, value in params_dict.items():
        print(f"  {key}: {value}")

    # Create model
    model = create_model(
        input_dim=X_train_scaled.shape[1],
        neurons_layer1=params_dict['neurons_layer1'],
        neurons_layer2=params_dict['neurons_layer2'],
        neurons_layer3=params_dict['neurons_layer3'],
        dropout_rate1=params_dict['dropout_rate1'],
        dropout_rate2=params_dict['dropout_rate2'],
        learning_rate=params_dict['learning_rate'],
        activation=params_dict['activation']
    )

    # Train model
    history = model.fit(
        X_train_scaled, y_train,
        epochs=20,
        batch_size=params_dict['batch_size'],
        validation_data=(X_val_scaled, y_val),
        callbacks=[early_stopping],
        verbose=1  # Set to 1 to see training progress
    )

    # Evaluate model on validation set
    val_pred = model.predict(X_val_scaled, verbose=0).flatten()
    val_mse = mean_squared_error(y_val, val_pred)
    val_rmse = np.sqrt(val_mse)
    val_mae = mean_absolute_error(y_val, val_pred)
    val_r2 = r2_score(y_val, val_pred)

    # Store results
    result = {
        **params_dict,
        'val_mse': val_mse,
        'val_rmse': val_rmse,
        'val_mae': val_mae,
        'val_r2': val_r2,
        'best_epoch': len(history.history['loss']) - early_stopping.patience
    }
    results.append(result)

    # Print current results
    print(f"  Validation RMSE: {val_rmse:.2f}")
    print(f"  Validation R²: {val_r2:.4f}")

    # Clean up to free memory
    del model
    tf.keras.backend.clear_session()

# End timing
end_time = time.time()
print(f"\nGrid search completed in {(end_time - start_time)/60:.2f} minutes")


Testing 128 hyperparameter combinations...

Training model 1/128 with parameters:
  neurons_layer1: 64
  neurons_layer2: 32
  neurons_layer3: 16
  dropout_rate1: 0.2
  dropout_rate2: 0.1
  learning_rate: 0.01
  activation: relu
  batch_size: 32
Epoch 1/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 14s 3ms/step - loss: 737001.0000 - val_loss: 302621.4062
Epoch 2/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 336425.7188 - val_loss: 348479.9062
Epoch 3/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 301216.7812 - val_loss: 389437.8750
Epoch 4/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 294093.4375 - val_loss: 371319.0000
Epoch 5/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 287535.1562 - val_loss: 366327.7500
Epoch 6/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 281497.0000 - val_loss: 349077.7500
Epoch 7/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 279184.5625 - val_loss: 319590.3125
Epoch 8/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss:

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1809/1809 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 1042893.9375 - val_loss: 272826.5000
Epoch 2/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - loss: 340454.4688 - val_loss: 266454.3438
Epoch 3/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 326275.0312 - val_loss: 258690.9688
Epoch 4/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 308966.0938 - val_loss: 269795.2812
Epoch 5/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 300816.6250 - val_loss: 258469.4375
Epoch 6/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 300148.0938 - val_loss: 261078.7188
Epoch 7/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - loss: 297966.2500 - val_loss: 257438.7969
Epoch 8/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 295371.3750 - val_loss: 253893.4844
Epoch 9/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 295197.9688 - val_loss: 252605.2031
Epoch 10/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 292753.6875 - val_loss: 256093.0469
Epoch 11/20
1809/1809 ━━━━

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3618/3618 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 1758524.0000 - val_loss: 262571.5625
Epoch 2/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 347308.7812 - val_loss: 259149.5000
Epoch 3/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 335826.7812 - val_loss: 256419.4375
Epoch 4/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 327912.8438 - val_loss: 256729.1875
Epoch 5/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 323944.8750 - val_loss: 256217.4688
Epoch 6/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 317302.8438 - val_loss: 256120.6875
Epoch 7/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 311767.1875 - val_loss: 254561.0312
Epoch 8/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 305690.9062 - val_loss: 253098.5000
Epoch 9/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 301224.8438 - val_loss: 252136.2031
Epoch 10/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 295784.4688 - val_loss: 251741.8750
Epoch 11/20
3618/36

...

3618/3618 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - loss: 1707170.5000 - val_loss: 263999.4062
Epoch 2/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 345811.4062 - val_loss: 257272.7812
Epoch 3/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 337461.7812 - val_loss: 258263.3906
Epoch 4/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 329634.5938 - val_loss: 258202.0781
Epoch 5/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 324958.4062 - val_loss: 256104.4219
Epoch 6/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 319589.8125 - val_loss: 258134.5469
Epoch 7/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - loss: 314207.2812 - val_loss: 253763.6250
Epoch 8/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 21s 3ms/step - loss: 310831.9688 - val_loss: 255655.1094
Epoch 9/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 19s 3ms/step - loss: 305682.9688 - val_loss: 252750.7812
Epoch 10/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - loss: 300677.8438 - val_loss: 254392.7656
Epoch 11/20
3618/36

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1809/1809 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 2610985.2500 - val_loss: 266026.6250
Epoch 2/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 361182.3125 - val_loss: 259525.4844
Epoch 3/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 344505.8125 - val_loss: 257594.8906
Epoch 4/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 339697.7188 - val_loss: 257343.3125
Epoch 5/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 330977.8125 - val_loss: 255034.7969
Epoch 6/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 327737.8438 - val_loss: 255220.3750
Epoch 7/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 324181.3125 - val_loss: 255803.7656
Epoch 8/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 321609.5000 - val_loss: 253380.8125
Epoch 9/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 314986.2500 - val_loss: 254480.1719
Epoch 10/20
1809/1809 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 311783.9375 - val_loss: 253298.6406
Epoch 11/20
1809/1809 ━━━

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3618/3618 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - loss: 715370.6875 - val_loss: 283017.1250
Epoch 2/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step - loss: 317295.0938 - val_loss: 275086.2812
Epoch 3/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 302986.6875 - val_loss: 279091.3750
Epoch 4/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 300022.3125 - val_loss: 268315.7500
Epoch 5/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 297338.7500 - val_loss: 283258.0625
Epoch 6/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 294625.1875 - val_loss: 268977.2500
Epoch 7/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 292033.4062 - val_loss: 266361.3125
Epoch 8/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 289161.8750 - val_loss: 258847.5938
Epoch 9/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 285603.1562 - val_loss: 255902.2031
Epoch 10/20
3618/3618 ━━━━━━━━━━━━━━━━━━━━ 11s 3ms/step - loss: 285903.2812 - val_loss: 255889.2344
Epoch 11/20
3618/361

In [ ]:
results_df = pd.DataFrame(results)

# Sort by validation RMSE (ascending)
results_df = results_df.sort_values('val_rmse')

# Save results to CSV
results_df.to_csv('hyperparameter_tuning_results.csv', index=False)
print("Hyperparameter tuning results saved to 'hyperparameter_tuning_results.csv'")

# Get best hyperparameters
best_params = results_df.iloc[0].to_dict()
print("\nBest hyperparameters:")
for key in keys:
    print(f"  {key}: {best_params[key]}")
print(f"Best validation RMSE: {best_params['val_rmse']:.2f}")
print(f"Best validation R²: {best_params['val_r2']:.4f}")

print("\n=== Training Final Model with Best Parameters ===\n")

# Create the best model
best_model = create_model(
    input_dim=X_train_scaled.shape[1],
    neurons_layer1=int(best_params['neurons_layer1']),
    neurons_layer2=int(best_params['neurons_layer2']),
    neurons_layer3=int(best_params['neurons_layer3']),
    dropout_rate1=best_params['dropout_rate1'],
    dropout_rate2=best_params['dropout_rate2'],
    learning_rate=best_params['learning_rate'],
    activation=best_params['activation']
)

# Add early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True
)

# Train the best model
history = best_model.fit(
    X_train_scaled, y_train,
    epochs=150,  # Longer training for final model
    batch_size=int(best_params['batch_size']),
    validation_data=(X_val_scaled, y_val),
    callbacks=[early_stopping],
    verbose=1
)

# Make predictions
y_train_pred = best_model.predict(X_train_scaled, verbose=0).flatten()
y_val_pred = best_model.predict(X_val_scaled, verbose=0).flatten()
y_test_pred = best_model.predict(X_test_scaled, verbose=0).flatten()


In [ ]:
def evaluate_model(y_true, y_pred, dataset_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"\n{dataset_name} Metrics:")
    print(f"Mean Squared Error: {mse:.2f}")
    print(f"Root Mean Squared Error: {rmse:.2f}")
    print(f"Mean Absolute Error: {mae:.2f}")
    print(f"R² Score: {r2:.4f}")

    return mse, rmse, mae, r2

# Evaluate the model
train_metrics = evaluate_model(y_train, y_train_pred, "Training")
val_metrics = evaluate_model(y_val, y_val_pred, "Validation")
test_metrics = evaluate_model(y_test, y_test_pred, "Test (2025 data)")

# Visualize the results
plt.figure(figsize=(18, 6))

# Plot training history
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot validation predictions
plt.subplot(1, 3, 2)
plt.scatter(y_val, y_val_pred, alpha=0.5)
plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'k--', lw=2)
plt.title('Validation: Predicted vs Actual')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.grid(True)

# Plot test predictions
plt.subplot(1, 3, 3)
plt.scatter(y_test, y_test_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2)
plt.title('Test (2025): Predicted vs Actual')
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.grid(True)

plt.tight_layout()
plt.savefig('tuned_model_results.png')
plt.show()


In [ ]:
def calculate_feature_importance(model, X, y, column_names):
    X_orig = X.copy()
    importance_scores = {}
    baseline_pred = model.predict(X_orig, verbose=0).flatten()
    baseline_mse = mean_squared_error(y, baseline_pred)

    for i, col in enumerate(column_names):
        X_shuffled = X_orig.copy()
        np.random.shuffle(X_shuffled[:, i])
        shuffled_pred = model.predict(X_shuffled, verbose=0).flatten()
        shuffled_mse = mean_squared_error(y, shuffled_pred)
        importance = shuffled_mse - baseline_mse
        importance_scores[col] = importance

    feature_importance_df = pd.DataFrame({
        'Feature': list(importance_scores.keys()),
        'Importance': list(importance_scores.values())
    })

    return feature_importance_df.sort_values('Importance', ascending=False)


In [ ]:
print("\nCalculating feature importance...")
feature_importance = calculate_feature_importance(best_model, X_test_scaled, y_test, X_test.columns)
print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

# Visualize feature importance
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(15)
plt.barh(top_features['Feature'], top_features['Importance'])
plt.xlabel('Importance')
plt.title('Top 15 Most Important Features')
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.show()

# Save the final model
best_model.save('tuned_rental_price_model.h5')
print("\nTuned model saved as 'tuned_rental_price_model.h5'")

# Save the scaler for future use
import joblib
joblib.dump(scaler, 'feature_scaler.pkl')
print("Feature scaler saved as 'feature_scaler.pkl'")


In [ ]:
print(f"Best hyperparameters:")
print(f"- Neurons in layer 1: {int(best_params['neurons_layer1'])}")
print(f"- Neurons in layer 2: {int(best_params['neurons_layer2'])}")
print(f"- Neurons in layer 3: {int(best_params['neurons_layer3'])}")
print(f"- Dropout rate 1: {best_params['dropout_rate1']}")
print(f"- Dropout rate 2: {best_params['dropout_rate2']}")
print(f"- Learning rate: {best_params['learning_rate']}")
print(f"- Activation function: {best_params['activation']}")
print(f"- Batch size: {int(best_params['batch_size'])}")
print(f"\nTest performance (2025 data):")
print(f"- RMSE: {test_metrics[1]:.2f}")
print(f"- R²: {test_metrics[3]:.4f}")

In [ ]:
Hi besties!